In [1]:
import spacy
import pickle
import Helpers
import numpy as np
import codecs

helper = Helpers.Exquisite_Corpse

encoder = spacy.load('en')

with open('data/final_lexicon.pkl', 'rb') as f:
    lexicon = pickle.load(f)

lexicon_lookup = helper.get_lexicon_lookup(lexicon)

In [2]:
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU

def create_model(seq_input_len, n_input_nodes, n_embedding_nodes, n_hidden_nodes, stateful=False, batch_size=None):

    input_layer = Input(batch_shape=(batch_size, seq_input_len), name='input_layer')
    
    embedding_layer = Embedding(input_dim=n_input_nodes,
                               output_dim=n_embedding_nodes,
                               mask_zero=True, name='embedding_layer')(input_layer)
    
    gru_layer1 = GRU(n_hidden_nodes,
                    return_sequences=True,
                    stateful=stateful,
                    name='hidden_layer1')(embedding_layer)
    
    gru_layer2 = GRU(n_hidden_nodes,
                    return_sequences=True,
                    stateful=stateful,
                    name='hidden_layer2')(gru_layer1)
    
    output_layer = TimeDistributed(Dense(n_input_nodes, activation="softmax"),
                                  name='output_layer')(gru_layer2)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    
    return model
    

Using TensorFlow backend.


In [3]:
predictor_model = create_model(seq_input_len=1,
                              n_input_nodes=len(lexicon) + 1,
                              n_embedding_nodes=250,
                              n_hidden_nodes = 400,
                              stateful=True,
                              batch_size=1)

predictor_model.load_weights('corpse_weights50.h5')



In [7]:
test_line = "In the morning, I travelled to a distant land where the creature and my sister were hiding."

words = [ [word.lower_ for word in encoder(test_line)] ]
words

[['in',
  'the',
  'morning',
  ',',
  'i',
  'travelled',
  'to',
  'a',
  'distant',
  'land',
  'where',
  'the',
  'creature',
  'and',
  'my',
  'sister',
  'were',
  'hiding',
  '.']]

In [8]:
model_input = helper.tokens_to_ids(words, lexicon)
model_input

[[1929,
  1896,
  221,
  1330,
  868,
  4084,
  3700,
  873,
  381,
  790,
  1412,
  1896,
  3760,
  4070,
  2921,
  547,
  3124,
  568,
  262]]

In [16]:
response = helper.generate_response(predictor_model, lexicon_lookup, model_input[0])
response

', and the wretch , was so been that the fire , and declined the society mingling of concern .'